## 1.  Import des données 

In [1]:
import csv
import numpy as np
%matplotlib inline
# pour ipyhton notebook 
import matplotlib as plt
import pylab as pl
import time

#### Implémentation  de deux fonctions utiles pour importer les fichiers :

In [2]:
def open_numeric(baseFileName, fieldnames=['user','movie','rating','datestamp'], delimiter='\t'):
    """
    fonction généralisée pour lire les fichiers numériques
    """
    with open(baseFileName, 'r') as f:
         reader = csv.DictReader(f, delimiter = delimiter, fieldnames=fieldnames)
         # create a dict out of reader, converting all values to integers
         return [dict([key, int(value)] for key, value in row.items()) for row in list(reader)]


In [3]:
def open_file(baseFileName, fieldnames=None , delimiter='|'):    
    """
    fonction généralisée pour lire les fichiers non numériques
    """   
    with open(baseFileName, 'r') as f:
         reader = csv.DictReader(f, delimiter = delimiter,fieldnames=fieldnames)
         return list(reader)

 ####  Création des deux directionnaires des ratings (20% et 80%) 
 ##### on travaille seulement sur u1.test et u1 base pour l'instant)

In [4]:
baseUserItem=open_numeric("u1.base")

In [5]:
testUserItem=open_numeric("u1.test")

## Informations sur les User / Occupation /Items (movies) / Genres 

In [6]:
User=open_file("u.user", fieldnames=['userID','age','gender','occupation','zipcode'])


In [7]:
with open('u.occupation', 'r') as f: #je n'ai pas réussi a utiliser la fonction généralisée, la doc de csv.reader n'est pas à jour
    reader = csv.reader(f)
    Occupation = list(reader)


In [8]:
Item = open_file("u.item", fieldnames=['movieID', 'Title', 'releaseDate', 'videoReleaseDate','IMDb URL', 'unknown',
                                       'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary',
                                       'Drama','Fantasy','Film-Noir', 'Horror', 'Musical', 'Mystery','Romance', 
                                       'Sci-Fi', 'Thriller', 'War', 'Western'])


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 2892: invalid continuation byte

In [ ]:
Genre=open_file("u.genre",fieldnames=['Genre:','Numero:'])


## 2.  Création de la matrice R 

#### R de taille M x N qui contient les notes des films Ni noté par le user Mi (ou 0 si film non noté)

M = nombre de users ; 
N = nb de films

In [ ]:
start1 = time.time()
start2 = time.time()

In [ ]:
# Remplir la matrice utilisateur-item 
NbUsers = len(User)
NbItems= len(Item)
R = np.zeros((NbUsers, NbItems))
for row in baseUserItem:
    R[row['user']-1,row['movie']-1] = row['rating']
pl.imshow(R,interpolation='none')

## 3.   Approximation bas-rang par SVD

 ###  Remplissage matrice RC (note moyenne film)


In [ ]:
def creation_RC(R):
    """
    :param R: matrice R utilisateur-item
    :return:  matrice RC des moyennes des films
    """
    NbUsers = R.shape[0]
    NbItems = R.shape[1]
    mean_items = [0 for row in range(NbItems)]  # liste des moyenne par colonne(film)
    for col in range(NbItems):
        rating_number = 0  # savoir le nombre d'éléments quon additionne pour diviser la moyenne
        for user_rating in R[:, col]:
            if user_rating != 0:  # on ne prend que les valeurs non nul
                mean_items[col] += user_rating
                rating_number += 1
        if rating_number != 0:  # il doit y avoir au moins une note par film
            mean_items[col] = mean_items[col] / rating_number
    RC = R.copy()
    for col in range(NbItems):
        for row in range(NbUsers):
            if RC[row, col] == 0:
                RC[row, col] = mean_items[col]
    return RC

In [ ]:
RC = creation_RC(R)
pl.imshow(RC,interpolation='none')

 ###  Remplissage matrice RR  (note moyenne user)


In [ ]:
def creation_RR(R):
    """
    :param R: matrice R utilisateur-item
    :return: matrice RC des moyennes utilisateurs
    """
    NbUsers = R.shape[0]
    NbItems = R.shape[1]
    mean_users = [0 for row in range(NbUsers)]  # liste des moyenne par lignes(users)
    for row in range(NbUsers):
        rating_number = 0  # savoir le nombre d'éléments qu'on additionne pour diviser la moyenne
        for user_rating in R[row, :]:
            if user_rating != 0:
                mean_users[row] += user_rating
                rating_number += 1
        if rating_number != 0:  # il doit y avoir au moins une note par film
            mean_users[row] = mean_users[row] / rating_number
    RR = R.copy()
    for row in range(NbUsers):
        for col in range(NbItems):
            if RR[row, col] == 0:
                RR[row, col] = mean_users[row]
    return RR  

In [ ]:
RR = creation_RR(R)
pl.imshow(RR,interpolation='none') 

In [ ]:
end2 = time.time()

## Décomposition en valeurs singulières des matrices RR et RC

In [ ]:
def svd_matrice(A):
    """ 
    A : matrice de taille MxN (ici M < N)
    Fonction qui réalise la décomposition en valeurs singulières de A tq : A = U x Sigma x VT
    Retourne :
    U : matrice de taille MxM unitaire
    Sigma : matrice  diagonal de taille MxN 
    VT : Transpose de la matrice V qui est de taille NxN unitaire
    """
    U, s, VT = np.linalg.svd(A, full_matrices=True)   # s est un vecteur de taille M (M < N) contenant les coeffs 
    S = np.zeros((A.shape[0], A.shape[1])) # on créé S une matrice de taille M x N
    m = min(A.shape[0], A.shape[1]) # ici M < N donc on aura m = M
    S[:m, :m] = np.diag(s) # On rempli S par la matrice carre diagonale MxM contenant les coeffs
    return U, S, VT
    
    

##### Calcul de $U_c, S_c, V'_c$  tel que : $ RC = U_cS_cV'_c$


In [ ]:
Uc, Sc, VTc = svd_matrice(RC)
print("A-t-on bien RC = Uc x Sc x VTc : ", np.allclose(RC, np.dot(Uc, np.dot(Sc, VTc))))

##### Calcul de $U_r, S_r, V'_r$  tel que : $ RC = U_rS_rV'_r$

In [ ]:
Ur, Sr, VTr = svd_matrice(RR)
print("A-t-on bien RR = Ur x Sr x VTr : ", np.allclose(RR, np.dot(Ur, np.dot(Sr, VTr))))

### Approximation bas-rang par SVD 

#####  Implémentation d'une fonction qui permmettra de calculer les approximations bas-rang de RC et de RR.

Le but est d'obtenir 30 matrices Rk = $U_kS_kV'_k$ pour k = 1,..,30.


In [ ]:
def approx_rang_k_svd_(U, S, VT):
    """
    :param U: matrice U résultant de la svd
    :param S: matrice S résultant de la svd
    :param VT: matrice VT résultant de la svd
    :return: un tuple contenant les listes de matrices Rk, Uk, Sk, VTk pour k = 1,..,30.
    """
    liste_Sk = []
    liste_Uk = []
    liste_VTk = []
    liste_Rk = []
    for k in range(1, 31):
        Sk = S[:k, :k]  # on remplie Sck (matrice carrée diagonale remplie des k premiers coeffs)
        liste_Sk.append(Sk)
        # matrice de rang k
        Uk = U[:, :k]
        VTk = VT[:k, :]
        Rk = np.dot(Uk, np.dot(Sk, VTk))
        liste_Uk.append(Uk)
        liste_VTk.append(VTk)
        liste_Rk.append(Rk)
    return liste_Rk, liste_Uk, liste_Sk, liste_VTk

In [ ]:
liste_RCk, liste_Uck, liste_Sck, liste_VTck = approx_rang_k_svd_(Uc, Sc, VTc)

In [ ]:
liste_RRk, liste_Urk, liste_Srk, liste_VTrk = approx_rang_k_svd_(Ur, Sr, VTr)

##### $U_k$ : taille M x k

###### $S_k$ : taille k x k

###### $VT_k$ : taille k x N 

### Matrices de features $X_k$ et $Y_k$

#### Interprétation dans le contexte utilisateur/film :  ####

Les matrices Xk et Yk sont des matrices de tendances et leur produit donne la matrice de prédiction : 

##### Xk (u,) :  indique la tendance pour l'utilisateurs u à suivre les k (premières) tendances
##### Yk (,i) :  indique la tendance pour le film i à suivre les k (premières) tendances


### Calcul de la  matrices de prédictions :

Les matrices de prédictions sont en fait tout simplement les matrices $R_k$ = $U_kS_kV'_k$ 


In [ ]:
def matrices_prediction(liste_Rk):
    """
    :param liste_Uk: liste des maatrice Rk
    :return: liste des 30 matrices de prédictions pour k = 1,..,30
    """
    liste_matrices_predictions = liste_Rk
    return liste_matrices_predictions

In [ ]:
liste_matrices_predictions_RC = matrices_prediction(liste_RCk)

In [ ]:
liste_matrices_predictions_RR = matrices_prediction(liste_RRk)

## 4. Qualité de la prédiction par SVD #

## Calcul des MAE pour $RC_k$ et $RR_k$ pour k=1,...,30 ##

Définition de la matrice Rtest qui contient les ratings réels :

In [ ]:
NbUsers = len(User)
NbItems= len(Item)
Rtest = np.zeros((NbUsers, NbItems))
for row in testUserItem:
    Rtest[row['user']-1,row['movie']-1] = row['rating']

In [ ]:
def calcul_MAE(Rtest, liste_matrices_prediction):
    """
    Calcule la MAE pour chaque matrice de prédiction
    :param Rtest: Matrice Test
    :param liste_matrices_prediction: liste des matrices de prédictions
    :return: liste des MAE pour ces matrices de matrices de prédictions
    """
    liste_MAE = []
    for k in range(0, 30):
        errorRating = []
        for i in range(0, Rtest.shape[0]):
            for j in range(0, Rtest.shape[1]):
                if Rtest[i][j] != 0:
                    errorRating.append(liste_matrices_prediction[k][i][j] - Rtest[i][j])
        liste_MAE.append(np.mean(np.abs(errorRating)))
    return liste_MAE

In [ ]:
liste_MAEc = calcul_MAE(Rtest, liste_matrices_predictions_RC)

In [ ]:
liste_MAEr = calcul_MAE(Rtest, liste_matrices_predictions_RR)

##### Implémantation d'une fonction pour trouver la meilleure matrice de prédictions (et à quelle approximation de bas-rang ellle correspond)

In [ ]:
def meilleure_matrice(liste_MAE):
    """
    Trouve quelle matrice donne les meilleures predictions et à quel K elle correspond
    :param liste_MAE: liste contenant les MAE des 30 matrices
    :return: la MAE la plus faible et pour quelle approximation de rang k on l'obtient
    """
    min_MAE = liste_MAE[0]
    index_min = 1
    for k in range(len(liste_MAE)):
        if liste_MAE[k] < min_MAE:
            min_MAE = liste_MAE[k]
            index_min = k + 1
    return min_MAE, index_min

In [ ]:
min_MAEc, index_minC = meilleure_matrice(liste_MAEc)
print("Pour RC : La meilleure matrice de prédiction a une MAE = ", min_MAEc,
      " et est obtenue pour l'approximation de rang k = ", index_minC)

In [ ]:
min_MAEr, index_minR = meilleure_matrice(liste_MAEr)
print("Pour RR : La meilleure matrice de prédiction a une MAE = ", min_MAEr,
      " et est obtenue pour l'approximation de rang k = ", index_minR)

In [ ]:
end1 = time.time()
temps_calcul_approx_bas_rang = end1 - start1
print(" Le temps nécessire pour déterminer les matrice de prédiction optimale (et leur MAE) est : ", round(temps_calcul_approx_bas_rang, 2), "secondes")

#### Comparaison avec les prédictions naïves, obtenues directement à partir de RC et de RR : 

In [ ]:
start22 = time.time()

In [ ]:
errorRating = []
for i in range(0, Rtest.shape[0]):
    for j in range(0, Rtest.shape[1]):
        if Rtest[i][j] != 0:
            errorRating.append(RC[i][j] - Rtest[i][j])
MAE_RC = (np.mean(np.abs(errorRating)))
print(" MAE pour la matrice de prédictions naïves RC : ", MAE_RC)

In [ ]:
errorRating = []
for i in range(0, Rtest.shape[0]):
    for j in range(0, Rtest.shape[1]):
        if Rtest[i][j] != 0:
            errorRating.append(RR[i][j] - Rtest[i][j])
MAE_RR = (np.mean(np.abs(errorRating)))
print(" MAE pour la matrice de prédictions naïves RR : ", MAE_RR)

In [ ]:
end22 = time.time()
temps_calcul_predictions_naives = (end2 - start2)  + (end22 - start22)
print(" Le temps nécessire pour déterminer les deux matrices de prédiction naïve RR et RC (et leur MAE)  est : ", round(temps_calcul_predictions_naives, 2), "secondes")

### Avantages de l'approximation bas-rang par rapport à l'utilisation de la matrice de rang plein :

$*$  Tout d'abord nous pouvons remarquer que les prédictions basées sur l'approximation de bas-rang  sont un peu plus précises que les prédictions naïves obtenues directements des matrices RR et RC. Nous pouvons aussi remarquer qu'il est préférable de travailler sur la matrice RC complétée par les notes moyennes des films. Les prédictions obtenues à partir de de cette matrice sont légèrement plus précises que celles obtenues avec RR (matrice des notes moyennes utilisateurs).

En effet l'approximation bas-rang permet de supprimer le bruit et en ne gardant que les informations importantes, nous  permmettant ainsi d'obtenir des prédictions plus proches de la réalité.


$*$ Mais ce gain en précision s'accompagne cependant d'un coût en stockage et en calcul plus important. En effet pour déterminer quelle approximation de bas-rang est la plus optimale, il est nécessaire de calculer, de les stocker et de calculer pour chacune d'elles la MAE associée. Même en se restreignant à $k \in [0,30]$, nous observons que le temps nécessaire pour déterminer la matrice de prédiction optimale par SVD puis approximation de bas-rang est 10 fois plus long.

###  Fonction générale pour tester la qualité de la prédiction par SVD chacun des cinq sets base/test disponibles 

(Fonction qui ré-utilise toutes les fonctions vues précédemment)

In [ ]:
def svd_prediction(numero_base_test):
    print("Jeu base/test n°", numero_base_test, " :")
    """
    numero_base_test : numero du set de base/test que l'on soouhaite utilisé
    return : La plus petite MAE et pour quel K on a donc obtenu les meilleures predictions
    """
    baseFile = "u" + str(numero_base_test) + ".base"
    testFile = "u" + str(numero_base_test) + ".test"
    baseUserItem = open_numeric(baseFile)
    testUserItem = open_numeric(testFile)

    # Matrice Test (pour évaluer la qualité de nos  prédictions)
    NbUsers = 943
    NbItems = 1682
    Rtest = np.zeros((NbUsers, NbItems))
    for row in testUserItem:
        Rtest[row['user'] - 1, row['movie'] - 1] = row['rating']

    # Remplir la matrice utilisateur-item R
    R = np.zeros((NbUsers, NbItems))
    for row in baseUserItem:
        R[row['user'] - 1, row['movie'] - 1] = row['rating']

    # Remplissage matrice RC (note moyenne film)
    RC = creation_RC(R)

    # Remplissage matrice RR (note moyenne user)
    RR = creation_RR(R)

    # On fait les svd :
    Uc, Sc, VTc = svd_matrice(RC)
    Ur, Sr, VTr = svd_matrice(RR)

    # Approximation de rang K (k = 1, .., 31) de la matrice RC
    liste_RCk, liste_Uck, liste_Sck, liste_VTck = approx_rang_k_svd_(Uc, Sc, VTc)

    # Approximation de rang K (k = 1, .., 31) de la matrice RR
    liste_RRk, liste_Urk, liste_Srk, liste_VTrk = approx_rang_k_svd_(Ur, Sr, VTr)

    # Calcul des matrices de predictions pour k = 1,...,30 :
    liste_matrices_predictions_RC = matrices_prediction(liste_RCk)
    liste_matrices_predictions_RR = matrices_prediction(liste_RRk)

    # Calcul des MAE :
    liste_MAEc = calcul_MAE(Rtest, liste_matrices_predictions_RC)
    liste_MAEr = calcul_MAE(Rtest, liste_matrices_predictions_RR)

    # Pour quel K a-t-on les meilleures prédictions :
    min_MAEc, index_minC = meilleure_matrice(liste_MAEc)
    print("Pour RCk : La meilleure matrice de prédiction a une MAE = ", min_MAEc,
          ", pour l'approximation de rang k = ", index_minC)

    min_MAEr, index_minR = meilleure_matrice(liste_MAEr)
    print("Pour RRk : La meilleure matrice de prédiction a une MAE = ", min_MAEr,
          ", pour l'approximation de rang k = ", index_minR)
    return (min_MAEc, index_minC, min_MAEr, index_minR)


In [ ]:
sum_min_MAEc = 0
sum_min_MAEr = 0
for i in range(1,6):
    A = svd_prediction(i)
    sum_min_MAEc += A[0]
    sum_min_MAEr += A[2]
    print("*" * 21)

In [ ]:
moyenne_min_MAEc = sum_min_MAEc/5
moyenne_min_MAEr = sum_min_MAEr/5

print("La MAE moyenne obtenue pour les matrice RCk optimales des 5 jeux base/test est : ", moyenne_min_MAEc )
print("La MAE moyenne obtenue sur les matrice RRk optimales des 5  les 5 jeux base/test est : ", moyenne_min_MAEr)

## 5. Question théorique

Soit $A$ une matrice réelle telle que : $ A \in M_{m,n} (\mathbb{R}$) 
Nous avons vu en cours que toute matrice admet une décomposition en valeurs singulières. 

Nous savons d'après le cours que $AA^*$ est hermitienne, et que $A^*A$ l'est aussi. Nous pouvons donc en déduire que les matrices $AA^*$ et $A^*A$ sont diagonalisables. 

Soit f un endomorphisme de $\mathbb{R}^m$ et g un endomorphisme de $\mathbb{R}^n$ tel que :  $M_B1(f) = AA^*$ et $M_B2(g) = A^*A$ avec $B1$ et $B2$ les bases canoniques usuelles.

Comme $AA^*$ (réciproquement $A^*A$) est diagonalisable, nous pouvons trouver une base de vecteurs propres de $\mathbb{R}^m$ (réciproquement $\mathbb{R}^n$).

Nous pouvons donc poser $U$ et $V$ dont les vecteurs colonnes sont les vecteurs propres de $AA^*$ et $A^*A$ tel que : 

$ AA^* = U \sum_1 U^t $ avec $\sum_1 = diag(Sp(AA^*))$ (les valeurs propres sont comptées avec multiplicité)

$ A^*A = V \sum_2 V^t $ avec $\sum_2 = diag(Sp(A^*A))$ (les valeurs propres sont comptées avec multiplicité)

Comme les vecteurs colonnes de $U$ et $V$ sont des vecteurs  d'espaces réels, alors ils sont à coefficients réels donc les matrices $U$ et $V$ sont à coefficients réels.

en posant $\sum$ la matrice diagonale qui contient les racines des valeurs propres non nulles de $AA^*$ comptées avec multiplicité et en prenant la matrice $U$ et la matrice $V$ trouvée précédemment,

On peut,  par identification, écrire la décomposition de $A$ en valeurs singulières de la façon suivante :

$A = U \sum V^* $ avec $U$ et $V$ réels



## 6. Problèmes des moindres carrés pour méthode alternée